## Part 1

### Introduction

#### The business problem I decided to tackle was to figure out where the locations of fitness centers are in correlation to city hall in Philadelphia, Pennsylvania. As a new citizen of this city and keen on getting into shape after a workday. Finding the closest fitness center is ideal for any member to where offices are located and places to live in the area. With the city as a whole as a populating for 1.5 million. Finding that fitness center close to where offices are located will be key in the search. 

### Data

#### The Data used was taken from foursquare location services with a radius of “5000” with the keyword of “fitness centers.” Using City hall as a center point (green dot) any locations that are considered fitness centers will be red dots.  

In [31]:
import numpy as np # library to handle data
!pip install geopy

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [32]:
# Get longitude and latitude for Philadelphia, Pennsylvania
address = 'Philadelphia, Pennsylvania'

geolocator = Nominatim(user_agent="philly_gyms")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Philadelphia, Pennsylvania is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Philadelphia, Pennsylvania is 39.9527237, -75.1635262.


In [33]:
# Set up Foursquare
CLIENT_ID = 'Z3VRLYKLCV1WKV4414VIXVG4MUFIKDBPZF2RGG31RU02EHBQ' # your Foursquare ID
CLIENT_SECRET = 'PLMCX5URJDC1QC0SN3QS0XLAX4X5OQER4GNAQSSPLJ5ORU3L' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Z3VRLYKLCV1WKV4414VIXVG4MUFIKDBPZF2RGG31RU02EHBQ
CLIENT_SECRET:PLMCX5URJDC1QC0SN3QS0XLAX4X5OQER4GNAQSSPLJ5ORU3L


In [34]:
search_query = 'Fitness Centers'
radius = 5000
print(search_query + ' Found')

Fitness Centers Found


In [35]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=Z3VRLYKLCV1WKV4414VIXVG4MUFIKDBPZF2RGG31RU02EHBQ&client_secret=PLMCX5URJDC1QC0SN3QS0XLAX4X5OQER4GNAQSSPLJ5ORU3L&ll=39.9527237,-75.1635262&v=20180605&query=Fitness Centers&radius=5000&limit=100'

In [36]:
results = requests.get(url).json()

In [37]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,4e3fd160483b04e17ab3d932,Fitness Center,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1598489598,False,1414 S Penn Sq,at The Residences at the Ritz Carlton,39.951488,-75.164925,"[{'label': 'display', 'lat': 39.95148849487305...",182,19102,US,Philadelphia,PA,United States,[1414 S Penn Sq (at The Residences at the Ritz...,NaN
1,4fe1061ee4b0cc64d6f9f9cf,Fitness Center,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1598489598,False,1421 Arch St,at Le Méridien Philadelphia,39.954933,-75.164253,"[{'label': 'display', 'lat': 39.95493316650390...",253,19102,US,Philadelphia,PA,United States,"[1421 Arch St (at Le Méridien Philadelphia), P...",NaN
2,4f6c5ff8e4b07a4bc3568a1e,Fitness Center,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",v-1598489598,False,1201 Market St,at Philadelphia Marriott Downtown,39.952059,-75.160670,"[{'label': 'display', 'lat': 39.95205877098671...",254,19107,US,Philadelphia,PA,United States,[1201 Market St (at Philadelphia Marriott Down...,NaN
3,4eb318e94690030a964f2c41,Fitness Center,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1598489598,False,1411 Walnut St,at The Exchange,39.949659,-75.164932,"[{'label': 'display', 'lat': 39.94965900000000...",361,19107,US,Philadelphia,PA,United States,"[1411 Walnut St (at The Exchange), Philadelphi...",NaN
4,50bcf1bde4b069826d82436c,Fitness Center,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",v-1598489598,False,1100 Arch St,at Hilton Garden Inn,39.953663,-75.158089,"[{'label': 'display', 'lat': 39.95366287231445...",475,19107,US,Philadelphia,PA,United States,"[1100 Arch St (at Hilton Garden Inn), Philadel...",NaN


In [38]:
# keep only columns that include Fitness center name, and anything that is associated with location in Center City Philadelphia
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Fitness Center,Gym / Fitness Center,1414 S Penn Sq,at The Residences at the Ritz Carlton,39.951488,-75.164925,"[{'label': 'display', 'lat': 39.95148849487305...",182,19102,US,Philadelphia,PA,United States,[1414 S Penn Sq (at The Residences at the Ritz...,4e3fd160483b04e17ab3d932
1,Fitness Center,Gym,1421 Arch St,at Le Méridien Philadelphia,39.954933,-75.164253,"[{'label': 'display', 'lat': 39.95493316650390...",253,19102,US,Philadelphia,PA,United States,"[1421 Arch St (at Le Méridien Philadelphia), P...",4fe1061ee4b0cc64d6f9f9cf
2,Fitness Center,Gym,1201 Market St,at Philadelphia Marriott Downtown,39.952059,-75.160670,"[{'label': 'display', 'lat': 39.95205877098671...",254,19107,US,Philadelphia,PA,United States,[1201 Market St (at Philadelphia Marriott Down...,4f6c5ff8e4b07a4bc3568a1e
3,Fitness Center,Gym / Fitness Center,1411 Walnut St,at The Exchange,39.949659,-75.164932,"[{'label': 'display', 'lat': 39.94965900000000...",361,19107,US,Philadelphia,PA,United States,"[1411 Walnut St (at The Exchange), Philadelphi...",4eb318e94690030a964f2c41
4,Fitness Center,Gym / Fitness Center,1100 Arch St,at Hilton Garden Inn,39.953663,-75.158089,"[{'label': 'display', 'lat': 39.95366287231445...",475,19107,US,Philadelphia,PA,United States,"[1100 Arch St (at Hilton Garden Inn), Philadel...",50bcf1bde4b069826d82436c


### Methodology 

##### Taking data above and plotting those data points on map in correlation to city hall

In [39]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=16)
# generate map centred around the City Hall

# add a red circle marker to represent the City Hall
folium.features.CircleMarker(
    [latitude, longitude],
    radius=15,
    color='Green',
    popup='City Hall',
    fill = True,
    fill_color = 'green',
    fill_opacity = 0.5
).add_to(venues_map)

# Gyms as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=3,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=1.0
    ).add_to(venues_map)

# display map
venues_map

### Discussion/Results

#### From the map generated above, you can see that there are dozens of options for citizens of Philadelphia have many options for fitness centers around city hall. I had initially thought the number of fitness centers would be less due to the limited number of residents around this area. As with most city centers, these areas are densely populated with offices. However with the high foot traffic these areas see on a daily basis. In the addition that people will have different workout routines such as before work, going on lunch breaks and right after work, before returning home. It would also be ideal to get the membership info of each of these fitness centers to get a better idea of which ones are popular. 

### Conclusion 

#### In this study, was to find “fitness centers” around a center location for any new or current resident of Philadelphia, Pennsylvania to find a place to get a workout around city hall. This map can be useful for those select individuals looking at a place like this. 